In [2]:
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

KAGGLE_KEY = os.getenv("KAGGLE_KEY")
KAGGLE_USERNAME = os.getenv("KAGGLE_USERNAME")

os.environ["KAGGLE_KEY"] = KAGGLE_KEY
os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME

In [4]:
# !pip install -q -U keras-nlp
# !pip install -q -U keras>=3

In [5]:
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [6]:
import keras
import keras_nlp

/Users/atharvabot7/Downloads/MLOPS-Tutorials/Fine-Tuning-Practice/gemmakeras/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import json
data = []
with open(r"databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)

        if features["context"]:
            continue
        
        template = "Instruction: \n {instruction} \n\n Response: {response}"
        data.append(template.format(**features))

In [8]:
data = data[:50]

In [9]:
data

['Instruction: \n Which is a species of fish? Tope or Rope \n\n Response: Tope',
 'Instruction: \n Why can camels survive for long without water? \n\n Response: Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time.',
 "Instruction: \n Alice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter? \n\n Response: The name of the third daughter is Alice",
 'Instruction: \n Who gave the UN the land in NY to build their HQ \n\n Response: John D Rockerfeller',
 'Instruction: \n Why mobile is bad for human \n\n Response: We are always engaged one phone which is not good.',
 'Instruction: \n What is a polygon? \n\n Response: A polygon is a form in Geometry.  It is a single dimensional plane made of connecting lines and any number of vertices.  It is a closed chain of connected line segments or edges.  The vertices of the polygon are formed where two edges meet.  Examples of polygons are hexagons, pentagons, and o

In [10]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [11]:
prompt = template.format(
    instruction = "What should I do on a trip to Europe?",
    response = "",
)

In [12]:
sampler = keras_nlp.samplers.TopKSampler(k=5,seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction: 
 What should I do on a trip to Europe? 

 Response: 
 1. You must know how to travel Europe by train, bus, airplane, and car, and how to buy a train ticket, a bus ticket, an airplane ticket, and a car ticket. 

 2. You must know how to buy a hotel room in Europe and where to find it. 

 3. You must know how to buy a tour ticket in Europe and where to find it. 

 4. You must know how to get around in Europe. 

 5. You must know what to do in Europe. 

 6. You must know the best places to go in Europe. 

 7. You must know the best times to go in Europe. 

 8. You must know the best ways to get from Europe to the U.S. 

 9. You must know what to do with your time in Europe. 

 10. You must know where to find good information about Europe, and where you can find a good guidebook to Europe. 

 11. You must know how to get to Europe by train, bus, airplane, and


## LoRA Fine Tuning

In [13]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [14]:
gemma_lm.preprocessor.sequence_length = 512

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)

In [15]:
optimizer.exclude_from_weight_decay(var_names=["bias","scale"])

In [16]:
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [17]:
gemma_lm.fit(data, epochs=1, batch_size=1)

50/50 ━━━━━━━━━━━━━━━━━━━━ 1153s 23s/step - loss: 0.4742 - sparse_categorical_accuracy: 0.4620


In [18]:
prompt = template.format(
    instruction = "What should I do on a trip to Europe?",
    response = "",
)

In [19]:
sampler = keras_nlp.samplers.TopKSampler(k=5,seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt,max_length=256))

Instruction: 
 What should I do on a trip to Europe? 

 Response: 
 1) If you have a trip, please book a hotel in advance to avoid the hotel is full. 
 2) If the hotel you have booked is not available, please book a hotel in advance. 
 3) If you are going to a big city, you should book a hotel in advance. 
 4) If you are going to a small town in Europe, you can book a hotel in advance or book it on the spot. 

 5) If you go to the countryside in France, you can rent a house or a room. 
 6) If you want to go to Paris, you should book a hotel in advance. 
 7) If you have a flight from London to Paris, you should buy a train ticket to Paris, because the train ticket is more expensive than the plane ticket. 
 8) If you want to visit the Alps, you should buy a train ticket to Geneva, because the price of the bus ticket to France is more expensive. 
 9) If you want to travel around Europe, you should buy a Eurail pass. 
 
